# QTM 151 Final Project
## Project Members: Tammy Dang, Bao Truong, Jocelyn Nguyen, Chau Anh Nguyen

### Data Description

The question we are examining with our dataset is:


*   **How does the frequency of pit stops correlate with the success rate of driver victories in Formula 1 racing?**

In order to examine this, we will be parsing and analyzing these datasets:

1.   pit_stops.csv
2.   driver_standings.csv

Each row represents a driver within an F1 race. This will be distinct based on raceID and driverId.

# Import libraries 

In [4]:
# "pandas" processes datasets
# "maplotlib.pyplot" generates graphs
# datetime packages for time series manipulation

import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as mdates
from datetime import date, time, datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import re

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)

# Import dataset
For our analysis we will be utilizing three datasets for F1 racing:


1.   pit_stops.csv
2.   driver_standings.csv


In [ ]:
# Read the CSV file
driver_standings = pd.read_csv("data_raw/driver_standings.csv")
pit_stops = pd.read_csv("data_raw/pit_stops.csv")


In [ ]:
# to see the data
display(driver_standings)
display(pit_stops)

# note: We will group them by raceId and driverId when merging

# MERGE
The following will be the merging process between driver_standings and pit_stops. They will be merged by grouping by raceId and driverId and inner joining the dataset. We will then use this to clean and create data descriptions to find out the dataset we can use for graphing, etc.

In [ ]:
#merge driver_standings and pit_stops by raceId and driverId. Store this in data frame, merged_f1
merged_f1 = pd.merge(driver_standings, pit_stops, on =['raceId', 'driverId'], how ='inner')
display(merged_f1)

# Data Description Diagnostic Commands
The following is code to understand the data we have better.

In [ ]:
#How many observations do we have in the dataset?
n_total = len(merged_f1)

#count non-missing observations
n_nonmiss_wins = merged_f1["wins"].count()  # there are no NAs.
n_nonmiss_stop = merged_f1["stop"].count()  # there are no NAs.
n_nonmiss_points = merged_f1["points"].count() # there are no NAs.

# get unique values of race and driverId
list_ids_race = pd.unique(merged_f1["raceId"])
list_ids_driver = pd.unique(merged_f1["driverId"])

# NOTE: please group by race and driverId. As there are different races within this dataset with a participation of d

# Data Cleaning

In [ ]:
# Use regular expression to clean "duration"
time_format = re.compile(r'^([0-5]?\d):([0-5]?\d)\.(\d{1,3})$') # regex of values like "16:44.718"
dropping_list = [] # create array of rows with the said regex

for index, row in merged_f1.iterrows(): #create for loop to iterate through merged_f1's rows
    if time_format.match(row['duration']):
        dropping_list.append(index)

merged_f1.drop(dropping_list, inplace=True)

# I realized that I don't actually need to do this because miliseconds is the cleaned version of duration
# But i'll keep it here so let me have my glory T.T. This took me 2 hours to figure out


In [ ]:
# Aggregate by raceId and driverId, looking at total stops, pit time, and wins of each driverId
agg_table = merged_f1.groupby(["raceId", "driverId"]).agg(total_stops = ("stop", "sum"),
                                               total_pit_time = ("milliseconds", "sum"),
                                               total_wins = ("wins", "sum"),
                                               total_points = ("points", "sum"),
                                               total_lap = ("lap", "sum")).reset_index()
agg_table

In [ ]:
# Create subplots to assess the relationship between, Total Pit Time, Total Stops, Total Wins
fig, list_subfig = plt.subplots(1, 3, figsize = (11,3))

list_subfig[0].scatter(agg_table["total_pit_time"], agg_table["total_stops"])
list_subfig[0].set_title("Total Pit Time vs. total_stops")
list_subfig[0].set_xlabel("Total Pit Time")
list_subfig[0].set_ylabel("Total Stops")

list_subfig[1].scatter(agg_table["total_pit_time"], agg_table["total_wins"])
list_subfig[1].set_title("Total Pit Time vs. Total Wins")
list_subfig[1].set_xlabel("Total Pit Time")
list_subfig[1].set_ylabel("Total Wins")

list_subfig[2].scatter(agg_table["total_stops"], agg_table["total_wins"])
list_subfig[2].set_title("Total Stops vs. Total Wins")
list_subfig[2].set_xlabel("Total Stops")
list_subfig[2].set_ylabel("Total Wins")

In [ ]:
# Linear Regression 1: Total Pit Time and Total Wins with summary
results_pit_time = smf.ols("total_wins ~ total_pit_time", data=agg_table).fit()
print(results_pit_time.params)
print(results_pit_time.summary())

# Linear Regression 2: Total Stops and Total Wins with summary
results_stops = smf.ols("total_wins ~ total_stops", data=agg_table).fit()
print(results_stops.params)
print(results_stops.summary())


# Find line of best fit parameters for Regression 1
m = results_pit_time.params["total_pit_time"]
b = results_pit_time.params["Intercept"]

# Find line of best fit parameters for Regression 2
m1 = results_stops.params["total_stops"]
b1 = results_stops.params["Intercept"]


#Subplot of Linear Regression 1 and 2 Visualization
fig, list_subfig = plt.subplots(1, 2, figsize = (9,3))

list_subfig[0].scatter(agg_table["total_pit_time"], agg_table["total_wins"])
list_subfig[0].plot(agg_table["total_pit_time"], m * agg_table["total_pit_time"] + b, color="red")
list_subfig[0].set_title("Total Stops vs. Total Wins")
list_subfig[0].set_xlabel("Total Pit Time")
list_subfig[0].set_ylabel("Total Wins")

list_subfig[1].scatter(agg_table["total_stops"], agg_table["total_wins"])
list_subfig[1].plot(agg_table["total_stops"], m1 * agg_table["total_stops"] + b1, color="red")
list_subfig[1].set_title("Linear Regression of Total Stops vs. Total Wins")
list_subfig[1].set_xlabel("Total Stops")
list_subfig[1].set_ylabel("Total Wins")

# Data Parsing/Analysis

# Data Findings

# RESULTS

# DISCUSSION

Tammy: By taking the csv datasets, driver_standings and pit_stops, we merged them by grouping driverId and raceId and stored them into variable merged_f1. Using this merged dataset, we created some data description diagnostic commands to be used for reference when parsing and cleaning the dataset.